In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install pyheif --no-cache-dir
%pip install pyheif pillow



In [ ]:
import os
import numpy as np
import face_recognition
import faiss
from PIL import Image, UnidentifiedImageError

DATASET_DIR = "dataset_images"

embeddings = []
metadata = []

# Function to convert HEIC to JPEG if required
def convert_heic_to_jpeg(image_path):
    try:
        from pillow_heif import register_heif_opener
        register_heif_opener()
        img = Image.open(image_path)
        new_path = image_path.rsplit(".", 1)[0] + ".jpg"
        img.save(new_path, "JPEG")
        print(f"Converted HEIC to JPEG: {new_path}")
        return new_path
    except ImportError:
        print("Install pillow-heif for HEIC support: pip install pillow-heif")
        return None
    except Exception as e:
        print(f"Error converting HEIC image: {image_path} - {e}")
        return None
    
for filename in os.listdir(DATASET_DIR):
    image_path = os.path.join(DATASET_DIR, filename)

    if not filename.lower().endswith((".jpg", ".jpeg", ".png", ".heic")):
        print(f"Skipping non-image file: {filename}")
        continue

    if filename.lower().endswith(".heic"):
        new_image_path = convert_heic_to_jpeg(image_path)
        if new_image_path is None:
            continue
        image_path = new_image_path

    try:
        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)

        if face_encodings:
            print(f"Detected {len(face_encodings)} face(s) in {filename}")
            embeddings.append(face_encodings[0])
            metadata.append(filename)
        else:
            print(f"No faces detected in {filename}. Skipping...")
    except UnidentifiedImageError:
        print(f"Skipping unreadable image file: {filename}")
    except Exception as e:
        print(f"Error processing file {filename}: {e}")

if embeddings:
    embeddings = np.array(embeddings)

    embedding_dim = 128
    index = faiss.IndexFlatL2(embedding_dim)
    index.add(embeddings)

    faiss.write_index(index, "face_index.bin")
    np.save("metadata.npy", np.array(metadata))

    print(f"Stored {len(embeddings)} embeddings in FAISS.")
else:
    print("No valid face embeddings found. Index creation skipped.")


In [ ]:
import cv2
import numpy as np
import face_recognition
import faiss
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt

index = faiss.read_index("face_index.bin")
metadata = np.load("metadata.npy")

THRESHOLD = 0.3
def display_image(image, title="Image"):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title, fontsize=14, color='blue')
    plt.title(title)
    plt.axis('off')
    plt.show()

def upload_image():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    if file_path:
        img = cv2.imread(file_path)
        return img
    return None

video_capture = cv2.VideoCapture(0)

print("Press 'C' to capture an image using webcam.")
print("Press 'U' to upload an image from your file system.")
print("Press 'Q' to quit.")

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    cv2.imshow("Video Feed", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord("c"):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        face_locations = face_recognition.face_locations(rgb_frame)

        if face_locations:
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            for face_encoding in face_encodings:
                face_encoding = np.array([face_encoding])
                distances, indices = index.search(face_encoding, k=len(metadata))

                matches = []
                for i, distance in enumerate(distances[0]):
                    if distance < THRESHOLD:
                        matches.append((metadata[indices[0][i]], distance))

                if matches:
                    person_matches = {}
                    for match_filename, match_distance in matches:
                        person_id = match_filename.split("_")[0]
                        if person_id not in person_matches:
                            person_matches[person_id] = []
                        person_matches[person_id].append(match_distance)

                    valid_persons = []
                    for person_id, distances in person_matches.items():
                        avg_distance = np.mean(distances)
                        if avg_distance < THRESHOLD:
                            valid_persons.append(person_id)

                    if valid_persons:
                        print(f"Found matches for: {', '.join(valid_persons)}")
                        for person_id in valid_persons:
                            for match_filename, match_distance in matches:
                                if match_filename.startswith(person_id):
                                    accuracy_score = (1 - match_distance) * 100
                                    print(f"  - {match_filename} (Distance: {match_distance}, Accuracy: {accuracy_score:.2f}%)")
                                    matched_image_path = f"dataset_images/{match_filename}"
                                    matched_img = cv2.imread(matched_image_path)
                                    if matched_img is not None:
                                        display_image(matched_img,f"Matched: {match_filename}\nAccuracy: {accuracy_score:.2f}%",)
                    else:
                        print("No relevant matches found. Please try again.")
                else:
                    print("No relevant matches found. Please try again.")
        else:
            print("No face detected in the captured image.")

    elif key == ord("u"):
        cv2.destroyAllWindows()
        uploaded_img = upload_image()
        if uploaded_img is not None:
            rgb_image = cv2.cvtColor(uploaded_img, cv2.COLOR_BGR2RGB)

            face_locations = face_recognition.face_locations(rgb_image)

            if face_locations:
                face_encodings = face_recognition.face_encodings(rgb_image, face_locations)
                for face_encoding in face_encodings:
                    face_encoding = np.array([face_encoding])
                    distances, indices = index.search(face_encoding, k=len(metadata))

                    matches = []
                    for i, distance in enumerate(distances[0]):
                        if distance < THRESHOLD:
                            matches.append((metadata[indices[0][i]], distance))

                    if matches:
                        person_matches = {}
                        for match_filename, match_distance in matches:
                            person_id = match_filename.split("_")[0]
                            if person_id not in person_matches:
                                person_matches[person_id] = []
                            person_matches[person_id].append(match_distance)

                        valid_persons = []
                        for person_id, distances in person_matches.items():
                            avg_distance = np.mean(distances)
                            if avg_distance < THRESHOLD:
                                valid_persons.append(person_id)

                        if valid_persons:
                            print(f"Found matches for: {', '.join(valid_persons)}")
                            for person_id in valid_persons:
                                for match_filename, match_distance in matches:
                                    if match_filename.startswith(person_id):
                                        print(f"  - {match_filename} (Distance: {match_distance})")
                                        matched_image_path = f"dataset_images/{match_filename}"
                                        matched_img = cv2.imread(matched_image_path)
                                        if matched_img is not None:
                                            display_image(matched_img, f"Matched: {match_filename}")
                        else:
                            print("No relevant matches found. Please try again.")
                    else:
                        print("No relevant matches found. Please try again.")
            else:
                print("No face detected in the uploaded image.")
        else:
            print("No image uploaded. Please try again.")

    elif key == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()
